In [9]:
import os
import sys
import numpy as np
import pandas as pd
from scipy import io
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import fiona
import geopandas as gpd
import netCDF4 as nc
from itertools import tee
import seaborn as sns
from matplotlib import cm
from matplotlib.colors import LogNorm, ListedColormap
import matplotlib.patches as mpatches
from matplotlib.collections import PatchCollection

import warnings
warnings.filterwarnings("ignore")

import read_data, concatenation, radar

In [15]:
class AnimatedPlot(object):
    """An animated scatter plot using matplotlib.animations.FuncAnimation."""
    def __init__(self, veh_table, radar0, radar1, xlabels, ylabels, wiperlabels, timestamps, ts_offset):
        self.xlabels = xlabels
        self.ylabels = ylabels
        self.wiperlabels = wiperlabels
        self.xbounds = (extent[0], extent[1])
        self.ybounds = (extent[2], extent[3])
        self.stream = self.data_stream(veh_table, radar0, radar1)
        self.timestamps = timestamps
        self.ts_offset = ts_offset

        # Setup the figure and axes...
        self.fig, self.ax = plt.subplots(2, figsize=(10,10))

    def setup_plot(self):
        """Initial drawing of the scatter plot."""
        sc, im0, im1, t = next(self.stream)
        sc_x = sc[:, 0]
        sc_y = sc[:, 1]
        sc_c = sc[:, 2]
        
        cmap = cm.gist_ncar
        my_cmap = cmap(np.arange(cmap.N))
        my_cmap[:,-1] = np.log10(np.linspace(1, 10, cmap.N))
        my_cmap = ListedColormap(my_cmap)
        vmin = 0
        vmax = 3

        # Draw backdrop
        # Subplot 0
        roads.plot(ax=self.ax[0], linewidth=0.5, color='k', alpha=0.5, zorder=1)
        self.scat0 = self.ax[0].scatter(sc_x, sc_y, c=sc_c, cmap='Greys', vmin=0, vmax=3, s=0, animated=True, zorder=4)
        self.ax[0].axis([self.xbounds[0], self.xbounds[1],
                      self.ybounds[0], self.ybounds[1]])
        self.im0 = self.ax[0].imshow(im0, animated=True, extent=extent, cmap=my_cmap, origin='lower',
           vmin=vmin, vmax=vmax, interpolation='nearest', zorder=3)
        self.text0 = self.ax[0].text(self.xbounds[0], self.ybounds[0], '',
                                 horizontalalignment='left', verticalalignment='bottom')
        self.ax[0].grid('off')
        self.ax[0].set_xticks([])
        self.ax[0].set_yticks([])
        self.ax[0].set_facecolor('0.86')
        self.ax[0].set_alpha(0.6)
        
        # Subplot 1
        roads.plot(ax=self.ax[1], linewidth=0.5, color='k', alpha=0.5, zorder=1)
        self.scat1 = self.ax[1].scatter(sc_x, sc_y, c=sc_c, cmap='Greys', vmin=0, vmax=3, s=9, animated=True, zorder=4)
        self.ax[0].axis([self.xbounds[0], self.xbounds[1],
                      self.ybounds[0], self.ybounds[1]])
        self.im1 = self.ax[1].imshow(im1, animated=True, extent=extent, cmap=my_cmap, origin='lower',
           vmin=vmin, vmax=vmax, interpolation='nearest', zorder=3)
        self.text1 = self.ax[1].text(self.xbounds[0], self.ybounds[0], '',
                                 horizontalalignment='left', verticalalignment='bottom')
        self.ax[1].grid('off')
        self.ax[1].set_xticks([])
        self.ax[1].set_yticks([])
        self.ax[1].set_facecolor('0.86')
        self.ax[1].set_alpha(0.6)

        red_patch = mpatches.Patch(facecolor='k', label='High', linewidth=1.2, edgecolor='k')
        yellow_patch = mpatches.Patch(facecolor='0.5', label='Medium', linewidth=1.2, edgecolor='k')
        cyan_patch = mpatches.Patch(facecolor='0.75', label='Low', linewidth=1.2, edgecolor='k')
        blue_patch = mpatches.Patch(facecolor='1.0', label='Off', linewidth=1.2, edgecolor='k')
        leg0 = self.ax[1].legend(handles=[red_patch, yellow_patch, cyan_patch, blue_patch],
                            frameon=True, fontsize=12, loc=2, title='Wiper intensity')
        leg0.get_frame().set_edgecolor('0.4')
        leg0.get_title().set_fontsize(12)
        leg0.get_title().set_fontweight('bold')
        leg0.get_frame().set_alpha(0.5)

        self.fig.colorbar(self.im0, ax=self.ax.ravel().tolist(), orientation='horizontal')

        # For FuncAnimation's sake, we need to return the artist we'll be using
        # Note that it expects a sequence of artists, thus the trailing comma        
        return self.scat0, self.scat1, self.im0, self.im1, self.text0, self.text1

    def data_stream(self, veh_table, radar0, radar1):
        """Generate data stream"""

        for ix, timestamp in enumerate(self.timestamps):
            # sc = veh_table.loc[start:end, [self.xlabels, self.ylabels, self.wiperlabels]].values
            sc = (veh_table[veh_table['Step'] == ix + self.ts_offset + 1])[[self.xlabels, self.ylabels, self.wiperlabels]].values
            im0 = radar0[ix + self.ts_offset, :, :]
            # Note that product is flipped
            im1 = radar1[:, :, ix + self.ts_offset]
            
            # s += 0.05 * (np.random.random(self.numpoints) - 0.5)
            # c += 0.02 * (np.random.random(self.numpoints) - 0.5)
            yield sc, im0, im1, timestamp

    def update(self, i):
        """Update the scatter plot."""
        sc, im0, im1, t = next(self.stream)

        sc_xy = sc[:, :2]
        sc_c = sc[:, 2]
        
        # Set x and y data...
        self.scat0.set_offsets(sc_xy)
        self.scat1.set_offsets(sc_xy)
        self.im0.set_array(im0)
        self.im1.set_array(im1)
        # Set sizes...
        # self.scat._sizes = 300 * abs(data[2])**1.5 + 100
        # Set colors..
        self.scat0.set_array(sc_c)
        self.scat1.set_array(sc_c)
        self.text0.set_text(str(t))
        self.text1.set_text(str(t))

        # We need to return the updated artist for FuncAnimation to draw..
        # Note that it expects a sequence of artists, thus the trailing comma.
        return self.scat0, self.scat1, self.im0, self.im1, self.text0, self.text1

    def show(self):
        plt.show()

In [3]:
# Get roadway shapefile
roads = gpd.read_file('../data/allroads_miv14a.shp')
wash_zips = [48105,48104,48106,48109,48108,49236,48190,48189,48115,48191,
49240,48198,48118,48197,48130,48137,48158,48160,48175,48176,48103,
]

wayne_zips = [
48277,48101,48111,48112,48120,48122,48124,
48126,48125,48128,48127,48134,48135,48138,48141,48150,48146,48152,48154,48164,
48167,48168,48173,48170,48174,48180,48184,48183,48186,48185,48188,48187,48192,
48195,48193,48202,48201,48204,48203,48206,48205,48208,48207,48210,48209,48212,
48211,48214,48213,48216,48215,48218,48217,48219,48222,48221,48224,48223,48226,
48225,48228,48227,48230,48229,48234,48236,48235,48238,48240,48239,48266,48265,
]
zips = wash_zips + wayne_zips
roads = roads[(roads['ZIPL'].isin(zips)) | (roads['ZIPR'].isin(zips))]
roads = roads.to_crs(epsg=4326)

In [4]:
# Read raw radar data
date = '20140811'
p_radar = read_data.radar_to_panel('../data/data_{0}.nc'.format(date), 'radar',
                                   dim_map={'lat' : 'latitude', 'lon' : 'longitude', 'time' : 'time'},
                                   time_unit='ns')

In [5]:
# Get vehicles
veh = pd.read_csv('../data/20140811_vehicle_filtered.csv', header=None)
veh.columns = ['Device', 'Trip', 'Latitude', 'Longitude', 'Wiper', 'Radar', 'Step', 'y', 'x']
veh['Time'] = veh['Step'].map(pd.Series(p_radar.axes[0]))
veh.set_index('Time', inplace=True)

In [6]:
# Get updated rainfall product
f1 = nc.Dataset('../data/product_20140811_agg.nc')
extent = (f1.variables['lon'][:].min(), f1.variables['lon'][:].max(),
          f1.variables['lat'][:].min(), f1.variables['lat'][:].max())

In [16]:
# Generate animation
a = AnimatedPlot(veh, p_radar.values, 6.285999774932861*f1.variables['combined'][:,:,:], 'Longitude', 'Latitude',
                    'Wiper', p_radar.axes[0].tolist()[150:], ts_offset=150)
ani = animation.FuncAnimation(a.fig, a.update, interval=1, frames=210 - 150, init_func=a.setup_plot)

In [17]:
# Write animation
Writer = animation.writers['ffmpeg']
writer = Writer(fps=10, metadata=dict(artist='MDB'), bitrate=1800)
ani.save('../img/movie_s1.mp4', writer=writer)